<a href="https://colab.research.google.com/github/Bartenderr/lotto_prediction/blob/v3.0/metro_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMPORT DEPENDENCIES FOR THIS ML PROJECT 

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, Dropout


Load data set into Data frame

In [2]:
df = pd.read_csv('/content/drive/MyDrive/metro_full_result.csv')
df

,Date,WN1,WN2,WN3,WN4,WN5,MAN1,MAN2,MAN3,MAN4,MAN5,Time
0,1/30/2017,42,56,47,43,51,NaN,NaN,NaN,NaN,NaN,19:45
1,02/03/2017,67,83,76,41,46,NaN,NaN,NaN,NaN,NaN,12:45
2,02/06/2017,5,12,61,89,28,NaN,NaN,NaN,NaN,NaN,19:45
3,02/10/2017,4,75,2,13,24,NaN,NaN,NaN,NaN,NaN,12:45
4,2/13/2017,64,8,77,69,22,NaN,NaN,NaN,NaN,NaN,19:45
...,...,...,...,...,...,...,...,...,...,...,...,...
617,01/09/2023,69,14,58,45,48,NaN,NaN,NaN,NaN,NaN,NaN
618,01/13/2023,39,89,51,11,53,NaN,NaN,NaN,NaN,NaN,NaN
619,01/16/2023,36,14,46,34,33,NaN,NaN,NaN,NaN,NaN,NaN
620,01/20/2023,35,37,5,15,39,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df.drop(["Date", "MAN1", "MAN2", "MAN3", "MAN4", "MAN5", "Time"], axis=1, inplace=True)

In [4]:
df.head()

,WN1,WN2,WN3,WN4,WN5
0,42,56,47,43,51
1,67,83,76,41,46
2,5,12,61,89,28
3,4,75,2,13,24
4,64,8,77,69,22


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 622 entries, 0 to 621
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   WN1     622 non-null    int64
 1   WN2     622 non-null    int64
 2   WN3     622 non-null    int64
 3   WN4     622 non-null    int64
 4   WN5     622 non-null    int64
dtypes: int64(5)
memory usage: 24.4 KB


In [6]:
df.tail()

,WN1,WN2,WN3,WN4,WN5
617,69,14,58,45,48
618,39,89,51,11,53
619,36,14,46,34,33
620,35,37,5,15,39
621,42,51,54,46,61


In [7]:
df.describe()

,WN1,WN2,WN3,WN4,WN5
count,622.000000,622.000000,622.000000,622.000000,622.000000
mean,44.538585,43.860129,45.935691,46.080386,44.540193
std,25.973372,25.918267,25.949767,25.620417,26.358622
min,1.000000,1.000000,1.000000,1.000000,1.000000
25%,22.000000,21.000000,24.000000,26.000000,21.000000
50%,45.000000,42.000000,48.000000,45.500000,46.000000
75%,66.750000,66.000000,67.750000,68.000000,67.750000
max,90.000000,90.000000,90.000000,90.000000,90.000000


In [8]:
df.shape

(622, 5)

re-scale data

In [9]:
scaler = StandardScaler().fit(df.values)
transformed_dataset = scaler.transform(df.values)
transformed_df = pd.DataFrame(data=transformed_dataset, index=df.index)

In [10]:
#Check_out scaled data 
transformed_df.head()

,0,1,2,3,4
0,-0.097817,0.468768,0.041047,-0.120328,0.245271
1,0.865482,1.511342,1.159490,-0.198454,0.055427
2,-1.523499,-1.230243,0.580985,1.676560,-0.628011
3,-1.562031,1.202431,-1.694468,-1.292212,-0.779886
4,0.749886,-1.384699,1.198057,0.895304,-0.855824


Define variables 

In [11]:
number_of_rows = df.values.shape[0]
number_of_rows

622

In [12]:
#AMOUNT OF GAMES WE NEED TO TAKE INTO CONSIDERATION
window_length = 7
window_length

7

In [13]:
#BALLS COUNT 

In [14]:
number_of_features = df.values.shape[1]
number_of_features

5

Create X and y for each rowin our scaled data.

In [15]:
#it should have the format for keras LTSM model(rows, window size, balls)
X = np.empty([number_of_rows - window_length, window_length, number_of_features], dtype=float)

y = np.empty([number_of_rows - window_length, number_of_features], dtype=float)

for i in range(0, number_of_rows-window_length):
  X[i] = transformed_df.iloc[i : i+window_length, 0 : number_of_features]
  y[i] = transformed_df.iloc[i+window_length : i+window_length+1, 0 : number_of_features]

In [16]:
X.shape

(615, 7, 5)

In [17]:
y.shape

(615, 5)

In [18]:
X[0]

array([[-0.09781665,  0.46876752,  0.0410472 , -0.12032845,  0.24527102],
       [ 0.86548224,  1.51134225,  1.15949041, -0.19845402,  0.0554271 ],
       [-1.52349902, -1.23024315,  0.5809853 ,  1.67655964, -0.62801101],
       [-1.56203098,  1.20243122, -1.69446812, -1.29221199, -0.77988615],
       [ 0.74988638, -1.38469866,  1.19805741,  0.89530395, -0.85582371],
       [ 0.82695029,  0.7004508 , -1.54020009,  0.23123661,  0.70089642],
       [ 0.59575855,  1.16381734, -1.46306608,  0.46561332, -1.53926182]])

In [19]:
y[0]

array([-1.52349902,  1.47272838,  1.04378938,  0.69999003, -1.27348034])

In [20]:
X[1]

array([[ 0.86548224,  1.51134225,  1.15949041, -0.19845402,  0.0554271 ],
       [-1.52349902, -1.23024315,  0.5809853 ,  1.67655964, -0.62801101],
       [-1.56203098,  1.20243122, -1.69446812, -1.29221199, -0.77988615],
       [ 0.74988638, -1.38469866,  1.19805741,  0.89530395, -0.85582371],
       [ 0.82695029,  0.7004508 , -1.54020009,  0.23123661,  0.70089642],
       [ 0.59575855,  1.16381734, -1.46306608,  0.46561332, -1.53926182],
       [-1.52349902,  1.47272838,  1.04378938,  0.69999003, -1.27348034]])

In [21]:
y[1]

array([-0.48313621, -1.30747091, -0.65315893,  0.11404826,  1.53620967])

In [22]:
#MODELING

In [23]:
model = Sequential()

In [24]:
model.add(Bidirectional(LSTM(240, input_shape =(window_length, number_of_features), return_sequences = True)))

In [25]:
#dropout layers in order to reduce overfitting
model.add(Dropout(0.2))

In [26]:
#Add a Second LSTM layer

In [27]:
model.add(Bidirectional(LSTM(240, input_shape =(window_length, number_of_features), return_sequences = True)))

In [28]:
#dropout again 
model.add(Dropout(0.2))

In [29]:
#Add a third LSTM layer

In [30]:
model.add(Bidirectional(LSTM(240, input_shape =(window_length, number_of_features), return_sequences = True)))

In [31]:
#Dropout again thrid time

In [32]:
model.add(Dropout(0.2))

In [33]:
#add a Fourth Layer of LSTM
model.add(Bidirectional(LSTM(240, input_shape =(window_length, number_of_features), return_sequences = False)))

In [34]:
#Add Dense Layer 
model.add(Dense(number_of_features))

In [35]:
#Compile the RNN

In [36]:
from tensorflow import keras
from tensorflow.keras.optimizers import Adam

model.compile(optimizer=Adam(learning_rate=0.0001), loss='mse', metrics=['accuracy'])


In [44]:
model.fit(x=X, y=y, batch_size=100, epochs=700, verbose=2)

Epoch 1/700
7/7 - 3s - loss: 0.0223 - accuracy: 0.8943 - 3s/epoch - 471ms/step
Epoch 2/700
7/7 - 3s - loss: 0.0215 - accuracy: 0.9366 - 3s/epoch - 473ms/step
Epoch 3/700
7/7 - 3s - loss: 0.0221 - accuracy: 0.9187 - 3s/epoch - 457ms/step
Epoch 4/700
7/7 - 3s - loss: 0.0230 - accuracy: 0.8911 - 3s/epoch - 461ms/step
Epoch 5/700
7/7 - 3s - loss: 0.0223 - accuracy: 0.9008 - 3s/epoch - 456ms/step
Epoch 6/700
7/7 - 3s - loss: 0.0238 - accuracy: 0.8992 - 3s/epoch - 457ms/step
Epoch 7/700
7/7 - 3s - loss: 0.0221 - accuracy: 0.9171 - 3s/epoch - 466ms/step
Epoch 8/700
7/7 - 3s - loss: 0.0207 - accuracy: 0.9073 - 3s/epoch - 449ms/step
Epoch 9/700
7/7 - 3s - loss: 0.0214 - accuracy: 0.9317 - 3s/epoch - 458ms/step
Epoch 10/700
7/7 - 3s - loss: 0.0223 - accuracy: 0.9089 - 3s/epoch - 461ms/step
Epoch 11/700
7/7 - 3s - loss: 0.0234 - accuracy: 0.8943 - 3s/epoch - 458ms/step
Epoch 12/700
7/7 - 3s - loss: 0.0232 - accuracy: 0.9057 - 3s/epoch - 450ms/step
Epoch 13/700
7/7 - 3s - loss: 0.0215 - accuracy: 

In [68]:
to_predict = df.tail(8) #This is the number of recently past games that the Model is using to predict this new game
to_predict  #You can tune this based on need try 11 then drop tail

,WN1,WN2,WN3,WN4,WN5
614,23,79,73,39,33
615,70,44,42,64,1
616,47,26,36,7,84
617,69,14,58,45,48
618,39,89,51,11,53
619,36,14,46,34,33
620,35,37,5,15,39
621,42,51,54,46,61


In [69]:
to_predict.drop([to_predict.index[-1]], axis=0, inplace=True) #commented out because I am predicting the next game 

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [60]:
to_predict


,WN1,WN2,WN3,WN4,WN5
614,23,79,73,39,33
615,70,44,42,64,1
616,47,26,36,7,84
617,69,14,58,45,48
618,39,89,51,11,53
619,36,14,46,34,33
620,35,37,5,15,39
621,42,51,54,46,61


In [71]:
prediction = df.tail(1)
#prediction = [] #use empty[] when prediction new games 
prediction

,WN1,WN2,WN3,WN4,WN5
621,42,51,54,46,61


In [72]:
to_predict = np.array(to_predict)
to_predict

array([[23, 79, 73, 39, 33],
       [70, 44, 42, 64,  1],
       [47, 26, 36,  7, 84],
       [69, 14, 58, 45, 48],
       [39, 89, 51, 11, 53],
       [36, 14, 46, 34, 33],
       [35, 37,  5, 15, 39]])

In [73]:
scaled_to_predict = scaler.transform(to_predict)
scaled_to_predict

array([[-0.82992381,  1.35688674,  1.04378938, -0.27657959, -0.43816709],
       [ 0.98107811,  0.00540098, -0.15178783,  0.69999003, -1.65316817],
       [ 0.09484313, -0.68964884, -0.38318988, -1.52658869,  1.49824089],
       [ 0.94254615, -1.15301539,  0.46528428, -0.04220288,  0.13136467],
       [-0.21341252,  1.74302553,  0.19531523, -1.37033756,  0.32120859],
       [-0.32900839, -1.15301539,  0.00248019, -0.47189351, -0.43816709],
       [-0.36754034, -0.26489618, -1.5787671 , -1.21408642, -0.21035439]])

In [74]:
y_pred = model.predict(np.array([scaled_to_predict]))

print('The predicted numbers in the last lottery game are:', scaler.inverse_transform(y_pred).astype(int)[0])

1/1 [==============================] - 0s 38ms/step
The predicted numbers in the last lottery game are: [41 52 54 47 62]


In [75]:
prediction = np.array(prediction)

print('The actual numbers in the last lottery game were:', prediction[0])

The actual numbers in the last lottery game were: [42 51 54 46 61]


# **PARAMETERS**

epochs - 700 
LSTM - 240
Dense - 240
drop out - 0.2

# **RESULT**


Correct predictions - 1
near - 4
best accuracy - 0.96

In [ ]:
##change data set next time -- Done
##play with the epochs try 519

In [ ]:
##increase the window_length

There're couple of options to increase the accuracy:

1) Increase the hidden layers in the LSTM node. 

and/or 

2) add another layer of the LSTM. Only 1 hidden layer may not be sufficient for the training of your data.

After making changes in the model as above, you will probably see the stabilization of the accuracy in some range. Based on that you can adjust the other parameters.

Another note: You will need to enable the embedding layer to convert words to vectors.